## stacking模型融合流程

#### 图一

<img src="../../../Other/img/stacking0.png" style="width:500px;height:300px;float:bottom">


#### 图二

<img src="../../../Other/img/stacking1.png" style="width:500px;height:300px;float:bottom">



假设:       
* X_train.shape = (10000, 10)  
* y_train.shape = (10000, )   
* X_test.shape = (3000, 10) 

***

model1进行5折交叉验证

第一折:
* valid_predict1.shape = (2000, )
* X_test_predict1.shape = (3000, 1)

第二折:
* valid_predict2.shape = (2000, )
* X_test_predict2.shape = (3000, 1)

第三折:
* valid_predict3.shape = (2000, )
* X_test_predict3.shape = (3000, 1)

第四折:
* valid_predict4.shape = (2000, )
* X_test_predict4.shape = (3000, 1)

第五折:
* valid_predict5.shape = (2000, )
* X_test_predict5.shape = (3000, 1)

将5次valid_predcit的结果进行拼接,令P1=(valid_predict1,valid_predict2,valid_predict3,valid_predict4,valid_predict5)

* 可以看出P1即为model1对X_train的预测

将5次X_test_predcit5的结果进行平均,令T1=(X_test_predict1 + X_test_predict2 + X_test_predict3 + X_test_predict4 + X_test_predict5)/5

* 可以看出T1即为model1对X_Test的预测

***

model2,model3,model4,model5同理

最后得到P1,P2,P3,P4,P5和T1,T2,T3,T4,T5

***

将第一层模型的输出训练和验证第二层模型

其中第一层模型训练第二层模型的输出为:X_train_stack = (P1, P2, P3, P4, P5)

其中第一层模型测试第二层模型的输出为:X_test_stack = (T1, T2, T3, T4, T5)

假设第二层模型为Logistics回归,则训练和验证过程为:
```python
clf_second = LogisticRegression(solver="lbfgs")
clf_second.fit(X_train_stack, y_train)
pred = clf_second.predict_proba(X_test_stack)[:,1]
roc_auc_score(y_test, pred)
```

总结:

1. 第一层的分类器要求本身就具有比较高的精度

2. 第一层分类器差异要大

3. 第二层不需要复杂的分类器,第二层使用逻辑回归的好处
    * 可以配合L1、L2正则化进一步防治过拟合
    * L1正则化可以有限的选择特征,从第一层中删除不必要的分类器
    * 逻辑回归的输出结果可以被理解为概率